In [ ]:
import warnings; warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import datetime
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
# import pmdarima
import pickle
import time
import os
from fbprophet import Prophet

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

pd.plotting.register_matplotlib_converters()

### Data Preparation

Load the dataframe outputed from data_munging.ipynb

In [ ]:
full_df = pickle.load(open('full_df', 'rb'))

Here, I load the calendar.csv and downcast the fields to save RAM usage. Dummy fields are created by combining the presence of events from $event\_name\_1$ and $event\_name\_2$ fields.

In [ ]:
calendar = pd.read_csv(f'calendar.csv')

In [ ]:
downcast_dict = {'wm_yr_wk': np.int16,
                'weekday': 'category',
                'wday': np.int16,
                'month': np.int16,
                'year': np.int16,
                'd': 'category',
                'snap_CA': np.uint8,
                'snap_TX': np.uint8,
                'snap_WI': np.uint8,
                'event_name_1': 'category',
                'event_type_1': 'category',
                'event_name_2': 'category',
                'event_type_2': 'category',
                'event': np.uint8}

event_types = calendar.event_type_1.unique()[1:] # remove null
for event_type in event_types:
    calendar['event_' + event_type.lower()] = ((calendar.event_type_1 == event_type) | (calendar.event_type_2 == event_type)).map({True: 1, False: 0})
    downcast_dict['event_' + event_type.lower()] = np.uint8
event_names = calendar.event_name_1.unique()[1:]
for event_name in event_names:
    calendar['event_' + event_name.lower()] = ((calendar.event_name_1 == event_name) | (calendar.event_name_2 == event_name)).map({True: 1, False: 0})
    downcast_dict['event_' + event_name.lower()] = np.uint8
calendar['event'] = (~calendar.event_name_1.isnull()).map({True: 1, False: 0})

calendar = calendar.astype(downcast_dict)

calendar['date'] = pd.to_datetime(calendar['date'])

calendar.set_index('date', inplace=True)

I then collect the dates for each holidays and store them in a dictionary.

In [ ]:
holidays = calendar[~calendar.event_name_1.isnull()]

In [ ]:
holidays_and_date = dict()
for event in holidays.event_name_1.unique():
    holidays_and_date[event.lower()] = holidays[(holidays.event_name_1 == event)
                                                | (holidays.event_name_2 == event)].index.values

Extract the department id from the $id$ field using regex.

In [ ]:
full_df['dept_id'] = full_df['id'].str.extract('([A-Z]+_\d+)')

a dataframe, by, is created by grouping the full_df according to $dept\_id$

In [ ]:
by = full_df.groupby(['dept_id']).resample('d').mean()
by = by.reset_index().set_index('date')

### Visualisation

The $demand$ of each department is plotted for the whole period.

In [ ]:
for dept_id in by.dept_id.unique():
    by[by.dept_id == dept_id].demand.plot(label=dept_id)
plt.legend()

These two are functions used to plot effects of holidays by department. The output will be demonstrated below for better understanding.

In [ ]:
def get_window(dt): #get the dates 14 days before and after the target date
    return pd.to_datetime(dt-np.timedelta64(14, 'D')), pd.to_datetime(dt+np.timedelta64(14, 'D'))

In [ ]:
events = holidays.event_name_1.unique()
def holidays_effect_on(dept_id):
    # the maximum number of years is 6 years for holidays
    fig, ax = plt.subplots(nrows=len(events)*2, ncols=3, figsize=(20,180)) # split the holidays into 2 by 3
    plt.tight_layout()

    for i in range(len(events)):
        event = events[i].lower()
        color = 'red' if i%2 else 'blue' # change the color for interleaving holidays to show each holiday clearly
        i *= 2
        dates = pd.to_datetime(holidays_and_date[event])
        for j in range(len(dates)):
            date = dates[j]
            lower_window, upper_window = get_window(date)
            window_df = by.query(f'dept_id == "{dept_id}"')[lower_window:upper_window]
            current_ax = ax[i+j//3][j%3]
            if len(window_df) < 23:
                current_ax.axes.xaxis.set_ticklabels([])
                current_ax.axes.yaxis.set_ticklabels([])
                continue
            window_df.demand.plot(ax=current_ax, color=color)
            title = event + ' ' + date.strftime('%Y-%m-%d') + ' wday ' + str(window_df.iloc[0].wday)
            current_ax.set_title(title)
            current_ax.axvline(x=date) # ouput a vertical line to clearly indicate which day the holiday occurs

See and list how many different deparments there are.

In [ ]:
df_full.dept_id.unique()

Plot the weekly $demand$ for the targetted department to compare how the holidays actually affect it.

In [ ]:
dept_id = "FOODS_1"
by.query(f'dept_id == {dept_id}').groupby('wday')['demand'].mean().plot()
plt.title(dept_id)

__Why *weekly* demand is plotted but not monthly or yearly?__ Because there is strong weekly seasonality, as shown in the acf_plot below (the notebook is run on kaggle because my local machine is not good enough to handle this large dataset, so there will not be ouput shown here, instead I will use some figure to demonstrate where clarification is needed)

In [ ]:
plot_acf(by.query(f'dept_id == {dept_id}').demand.dropna())

![acf plot showing strong weekly seasonality](figures/acf_plot.png)

Use the method written above, part of the output is demonstrated as below.

In [ ]:
holidays_effect_on(dept_id)

![demonstration](figures/effect-of-holiday-by-department.png)

e.g. We can see that there is a surge of demand every year before the easter day, while martinlutherkingday doesn't have much impact on the seasonality